In [4]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import RFormula
import pandas as pd
import click


In [5]:
spark=SparkSession.builder.appName("App").getOrCreate()

In [6]:
file_path="hdfs://localhost:9000/user/hadoop/kaggel/housing.csv"
airbnbDF = spark.read.csv(file_path, header=True, inferSchema=True)
airbnbDF.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [7]:
trainDF,testDF=airbnbDF.randomSplit([.8,.2],seed=42)
trainDF = trainDF.dropna(subset=["total_bedrooms", "median_house_value"])
print(f"there are{trainDF.count()} rows in the training set and {testDF.count()} rows in test set")     

there are16403 rows in the training set and 4080 rows in test set


In [8]:
vecAssembler= VectorAssembler(inputCols=["total_bedrooms"],outputCol="features")
vecTrainDF=vecAssembler.transform(trainDF)
vecTrainDF.select("total_bedrooms","features","median_house_value").show(5)

+--------------+--------+------------------+
|total_bedrooms|features|median_house_value|
+--------------+--------+------------------+
|         300.0| [300.0]|           94600.0|
|         552.0| [552.0]|           85800.0|
|         528.0| [528.0]|           79000.0|
|         394.0| [394.0]|          111400.0|
|         419.0| [419.0]|           76100.0|
+--------------+--------+------------------+
only showing top 5 rows



In [9]:
lr = LinearRegression(featuresCol="features", labelCol="median_house_value")
lrmodel = lr.fit(vecTrainDF)
m = round(lrmodel.coefficients[0], 2)
b = round(lrmodel.intercept, 2)
print(f"the formula for linera regerrsion line is price = {m}*bedrooms + {b}")


the formula for linera regerrsion line is price = 14.89*bedrooms + 197728.32


In [10]:
pipeline=Pipeline(stages=[vecAssembler, lr])
pipelineModel=pipeline.fit(trainDF)
predDF=pipelineModel.transform(testDF)
predDF.select("total_bedrooms","features","median_house_value","prediction").show()

+--------------+--------+------------------+------------------+
|total_bedrooms|features|median_house_value|        prediction|
+--------------+--------+------------------+------------------+
|         531.0| [531.0]|          103600.0|205633.60243618884|
|         453.0| [453.0]|          106700.0|204472.37429262503|
|         616.0| [616.0]|           73200.0|206899.04336186734|
|        1056.0|[1056.0]|           90100.0| 213449.5610947914|
|         364.0| [364.0]|           67000.0|203147.38320573812|
|         309.0| [309.0]|           86400.0| 202328.5684891226|
|         479.0| [479.0]|           70500.0| 204859.4503404796|
|         283.0| [283.0]|           85100.0|  201941.492441268|
|         618.0| [618.0]|           80500.0|206928.81844247153|
|         214.0| [214.0]|           96000.0| 200914.2521604231|
|         385.0| [385.0]|           75500.0| 203460.0215520822|
|         324.0| [324.0]|           75000.0| 202551.8815936541|
|         511.0| [511.0]|          10060

In [11]:
trainDF,testDF=airbnbDF.randomSplit([.8,.2],seed=42)
trainDF = trainDF.dropna(subset=["total_bedrooms","longitude","latitude", "median_house_value"])
print(f"there are{trainDF.count()} rows in the training set and {testDF.count()} rows in test set")

there are16403 rows in the training set and 4080 rows in test set


In [12]:
vecAssembler= VectorAssembler(inputCols=["total_bedrooms","longitude","latitude"],outputCol="features")
vecTrainDF=vecAssembler.transform(trainDF)
vecTrainDF.select("total_bedrooms","longitude","latitude","features","median_house_value").show(5)

+--------------+---------+--------+--------------------+------------------+
|total_bedrooms|longitude|latitude|            features|median_house_value|
+--------------+---------+--------+--------------------+------------------+
|         300.0|  -124.35|   40.54|[300.0,-124.35,40...|           94600.0|
|         552.0|   -124.3|    41.8| [552.0,-124.3,41.8]|           85800.0|
|         528.0|  -124.27|   40.69|[528.0,-124.27,40...|           79000.0|
|         394.0|  -124.26|   40.58|[394.0,-124.26,40...|          111400.0|
|         419.0|  -124.25|   40.28|[419.0,-124.25,40...|           76100.0|
+--------------+---------+--------+--------------------+------------------+
only showing top 5 rows



In [13]:
lr = LinearRegression(featuresCol="features", labelCol="median_house_value")
lrmodel = lr.fit(vecTrainDF)

coeffs = lrmodel.coefficients
intercept = round(lrmodel.intercept, 2)
m1 = round(coeffs[0], 2)  
m2 = round(coeffs[1], 2)  
m3 = round(coeffs[2], 2)  

print(f"The formula for the multiple linear regression line is:")
print(f"price = {m1}*bedrooms + {m2}*longitude + {m3}*latitude + {intercept}")

The formula for the multiple linear regression line is:
price = 15.2*bedrooms + -71302.71*longitude + -69323.48*latitude + -5857591.92


In [14]:
pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(trainDF)
predDF = pipelineModel.transform(testDF)
predDF.select("total_bedrooms", "longitude", "latitude", "features", "median_house_value", "prediction").show()

+--------------+---------+--------+--------------------+------------------+------------------+
|total_bedrooms|longitude|latitude|            features|median_house_value|        prediction|
+--------------+---------+--------+--------------------+------------------+------------------+
|         531.0|   -124.3|   41.84|[531.0,-124.3,41.84]|          103600.0|112909.97135147732|
|         453.0|  -124.23|   40.54|[453.0,-124.23,40...|          106700.0|196854.03667662852|
|         616.0|  -124.23|   41.75|[616.0,-124.23,41...|           73200.0|115449.53553005122|
|        1056.0|  -124.19|   40.73|[1056.0,-124.19,4...|           90100.0|189993.51886608265|
|         364.0|  -124.18|   40.78|[364.0,-124.18,40...|           67000.0| 175298.8414737638|
|         309.0|  -124.17|   40.62|[309.0,-124.17,40...|           86400.0|184841.80373211857|
|         479.0|  -124.17|   40.79|[479.0,-124.17,40...|           70500.0| 175640.0936219655|
|         283.0|  -124.16|    40.6|[283.0,-124.16,